In [1]:
import torch 
import numpy as np
from utils.geo_utils import normalize_M
from datasets.Euclidean import test_euclidean_dataset

In [2]:
test_euclidean_dataset("120_frame_test")

Test Ps and M
Reprojection Error: Mean = 3396.150607929185, Max = 50596364.921368115
Test Ps and M_gt
Reprojection Error: Mean = 0.46923232111596175, Max = 5207.450248752113


In [29]:
gtR =  np.load("../results/Optimization_Euc/2024_04_25_11_03_55/OPTIMIZATION/UZH_Drone_IMU/cameras/Rs_gt.npy")
gtt =  np.load("../results/Optimization_Euc/2024_04_25_11_03_55/OPTIMIZATION/UZH_Drone_IMU/cameras/ts_gt.npy")[:, :, None]
np.savetxt('../results/Optimization_Euc/2024_04_25_11_03_55/OPTIMIZATION/UZH_Drone_IMU/cameras/gt.txt', np.concatenate([gtR, gtt], axis=2).reshape(-1, 12))

In [25]:
traj =  np.load("../results/Optimization_Euc/2024_04_25_11_03_55/OPTIMIZATION/UZH_Drone_IMU/cameras/Ps.npy")
print(traj.shape)
np.savetxt('../results/Optimization_Euc/2024_04_25_11_03_55/OPTIMIZATION/UZH_Drone_IMU/cameras/pred.txt', traj.reshape(-1, 12))

(110, 3, 4)


NameError: name 'trajreshape' is not defined

In [32]:
from pytorch3d import transforms as py3d_trans
from utils import geo_utils
from torch import nn
from torch.nn import functional as F

class GTLoss(nn.Module): # Not tested
    def __init__(self):
        super().__init__()
        self.calibrated = True

    def forward(self, pred_cam, data, epoch=None):
        # Get orientation
        Vs_gt = data.y[:, 0:3, 0:3].inverse().transpose(1, 2)
        if self.calibrated:
            #Rs_gt = geo_utils.rot_to_quat(torch.bmm(data.Ns_invT, Vs_gt).transpose(1, 2))
            Rs_gt = py3d_trans.matrix_to_quaternion(torch.bmm(data.Ns_invT, Vs_gt).transpose(1, 2))

        # Get Location
        t_gt = -torch.bmm(data.y[:, 0:3, 0:3].inverse(), data.y[:, 0:3, 3].unsqueeze(-1)).squeeze()

        # Normalize scene by points
        # trans = pts3D_gt.mean(dim=1)
        # scale = (pts3D_gt - trans.unsqueeze(1)).norm(p=2, dim=0).mean()

        # Normalize scene by cameras
        trans = t_gt.mean(dim=0)
        scale = (t_gt - trans).norm(p=2, dim=1).mean()

        t_gt = (t_gt - trans)/scale
        new_Ps = geo_utils.batch_get_camera_matrix_from_Vt(Vs_gt, t_gt)

        Vs_invT = pred_cam["Ps_norm"][:, 0:3, 0:3]
        Vs = torch.inverse(Vs_invT).transpose(1, 2)


        ts = torch.bmm(-Vs, pred_cam["Ps_norm"][:, :, 3].unsqueeze(dim=-1)).squeeze() # [:, [2,1,0], 3]

        # Translation error
        translation_err = (t_gt - ts).norm(p=2, dim=1)

        # Calculate error
        if self.calibrated:
            # Rs = geo_utils.rot_to_quat(torch.bmm(data.Ns_invT, Vs).transpose(1, 2))
            Rs = py3d_trans.matrix_to_quaternion(torch.bmm(data.Ns_invT, Vs).transpose(1, 2))

            orient_err = (Rs - Rs_gt).norm(p=2, dim=1)
        else:
            Vs_gt = Vs_gt / Vs_gt.norm(p='fro', dim=(1, 2), keepdim=True)
            Vs = Vs / Vs.norm(p='fro', dim=(1, 2), keepdim=True)
            orient_err = torch.min((Vs - Vs_gt).norm(p='fro', dim=(1, 2)), (Vs + Vs_gt).norm(p='fro', dim=(1, 2)))

        orient_loss = orient_err.mean()
        tran_loss = translation_err.mean()
        loss = orient_loss + tran_loss

        if epoch is not None and epoch % 1000 == 0:
            # Print loss
            print("loss = {}, orient err = {}, trans err = {}".format(loss, orient_loss, tran_loss))

        return loss

In [33]:
gtloss = GTLoss()

TypeError: __init__() missing 1 required positional argument: 'conf'

In [6]:
data = np.load("../datasets/Euclidean/Drinking Fountain Somewhere In Zurich.npz")
M = torch.tensor(data['M']) # [M*2, N] points
Ns = torch.tensor(data['Ns']) # [M, 3, 3] matrices
normalize_M(M, Ns)

tensor([[[ 0.2467,  0.0810],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         ...,
         [ 0.0500, -0.0243],
         [ 0.0000,  0.0000],
         [-0.1532, -0.2354]],

        [[ 0.2202,  0.0801],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.2026,  0.0781],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        ...,

        [[ 0.0000,  0.0000],
         [-0.2925, -0.1265],
         [ 0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [-0.1445, -0.2571]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.0000,  0.0000],
       

In [ ]:
embed_kwargs = {
    'include_input': True,
    'input_dims': 2,
    'max_freq_log2': 50 - 1,
    'num_freqs': 50,
    'log_sampling': True,
    'periodic_fns': [torch.sin, torch.cos],
}

embedder = Embedder(**embed_kwargs)

embedder.embed(torch.ones(10,2)).shape